In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Tabular Playground Series - Sep 2021   ---    use gpu!.



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from time import time 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
import warnings
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import explained_variance_score
warnings.filterwarnings('ignore')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
#--- tpg 2021 sep ---  data  preprosesing.  ----

import pandas as pd
train =pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
train = train.iloc[:900000]
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
train = train.drop('id',axis=1)
test = test.drop('id',axis=1)

y_train = train['claim']
y_train = y_train.values
y_train = y_train.reshape(-1, 1)
X_train = train.drop('claim',axis=1)

from sklearn.impute import SimpleImputer
import numpy as np
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp_mean.fit_transform(X_train)
test = imp_mean.transform(test)

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3) 

In [ ]:
!pip install torchvision
import torch

In [ ]:
!pip install pytorch-tabnet

In [ ]:
seed = 35

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.metrics import Metric
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import seaborn as sns
sns.set()

pretrain.

In [ ]:
tabnet_params = dict(
    n_d = 16,
    n_a = 16,
    n_steps = 5,
    gamma = 1.3,
    lambda_sparse = 1e-3,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = seed,
    verbose = 10
)

In [ ]:
# gpu ON.  10 minuts

unsupervised_model = TabNetPretrainer(**tabnet_params)

In [ ]:
unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_valid],
    pretraining_ratio=0.8,
)

tabnet --- train

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
max_epochs = 200
Bs = 1024
clf = TabNetRegressor(  verbose = 10 ,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, 
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax'
                     )

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle','rmse'],
    max_epochs=max_epochs,
    patience=15,
    batch_size=Bs, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    from_unsupervised=unsupervised_model
)

In [ ]:
print(f"BEST VALID SCORE  : {clf.best_cost}")

In [ ]:
#test_pred= X_test1.to_numpy()
test_pred = test
predictions =  clf.predict(test_pred)

In [ ]:
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
predictions = pd.DataFrame(predictions) # from array to dataframe
sample['claim'] = predictions
sample.to_csv('tabnet_with_pretrain_submission.csv',index=False)
sample